In [ ]:
import os
from os import path
from config import DATA_PATH
import pandas as pd
import numpy as np
import spatialdata as sd
import os.path as osp
from skimage.measure import regionprops

from data import patch_john, patch_harry

In [ ]:
sdata = sd.read_zarr(osp.join(DATA_PATH, "UC6_I.zarr/UC6_I.zarr"))
